In [1]:
%matplotlib inline

import os
import json 
import requests

import pandas as pd
import numpy as np

In [3]:
## Download Data
BASE = os.path.abspath(os.path.join('.', 'data'))
URL  = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"

def fetch_file(url=URL, name='online_retail.xlsx', base=BASE):
    """
    Fetch the data from the url and save it to the name in the base directory.
    """
    # Construct the path to save the data to 
    path = os.path.join(base, name)
    if os.path.exists(path): raise Exception("Data is already donwloaded!")
    
    # Fetch the URL qith requests.py
    response = requests.get(url)
    response.raise_for_status() 
    
    with open(path, 'w') as f:
        f.write(response.content)

# Uncomment below if you want to fetch the data 
# fetch_file()

In [4]:
DATA = os.path.join(BASE, 'online_retail.xlsx')

## Load the data frame for data exploration 
df = pd.read_excel(DATA)

In [5]:
df.Description = df.Description.replace(np.nan,' ', regex=True)
df.Description = df.Description.astype(unicode)

In [6]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin 

class ColumnSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, column):
        self.column = column 
        
    def fit(self, X, y=None):
        return self 

    def transform(self, df):
        """
        Expects a dataframe that contains the specified column. Returns only that column.
        """
        if self.column not in df.columns:
            raise Exception("Could not extract the '{}' column!".format(self.column))
        return df[self.column].values

In [ ]:
class LabelEncoder(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        self.classes_ = np.unique(X)
        return self

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X)

    def transform(self, X):
        classes = np.unique(X)
        if len(np.intersect1d(classes, self.classes_)) < len(classes):
            diff = np.setdiff1d(classes, self.classes_)
            raise ValueError("y contains new labels: %s" % str(diff))
        return np.searchsorted(self.classes_, X)

    def inverse_transform(self, X):
        diff = np.setdiff1d(X, np.arange(len(self.classes_)))
        if diff:
            raise ValueError("y contains new labels: %s" % str(diff))
        X = np.asarray(X)
        return self.classes_[X]
    

class TwoD(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self 

    def transform(self, X):
        return np.asarray([
            [x] for x in X
        ])

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import Imputer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.neighbors import NearestNeighbors


# Set up a feature extraction pipeline. 
model = Pipeline([
    ('columns', FeatureUnion(
        transformer_list = [
            
            ('description', Pipeline([
                ('select', ColumnSelector('Description')), 
                ('tfidf', TfidfVectorizer()), 
            ])), 
                    
            ('country', Pipeline([
                ('select', ColumnSelector('Country')), 
                ('vect', LabelEncoder()),
                ('2d', TwoD()),
            ])), 
                    
            ('price', Pipeline([
                ('select', ColumnSelector('UnitPrice')), 
                ('2d', TwoD()),
                ('impute', Imputer(np.nan, 'mean')),
            ])),
                    
            ('customer', Pipeline([
                ('select', ColumnSelector('CustomerID')),
                ('2d', TwoD()),
                ('impute', Imputer(np.nan, 'mean')),
            ])),
                    
            
        ],
                
        transformer_weights = {
            'description': 0.4, 
            'country': 0.1, 
            'price': 0.1, 
            'customer': 0.4,
        },
    )),
        
    ('neighbors', NearestNeighbors(k=15)), 
    
])

model = model.fit(df)

In [ ]:
import pickle 

model_path = os.path.join(BASE, 'neighbors.pickle')
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

In [ ]:
model.steps[-1][-1].kneighbors_graph()